# <font color='gray'> ECG View for gen3_sdk</font>



<font color='gray'>Import the gen3_sdk library from the path and import all the dependancies that are going to be used in the project like serial library for connection, matplotlib for plotting </font>

In [ ]:
import sys,os,serial,threading, time,struct,glob
if sys.platform == "linux" or sys.platform == "linux2":
    sys.path.append('./../../../bin/linux/python/')
elif sys.platform == "darwin":
    sys.path.append('./../../../bin/macOS/python/')
elif sys.platform == "win32":
    if ((struct.calcsize("P") * 8) == 64):
        sys.path.append('./../../../bin/windows/x64/python/')  # 32 bit python
    else:
        sys.path.append('./../../../bin/windows/Win32/python/')  # 64 bit python
elif sys.platform == "win64":
    sys.path.append('./../../../bin/windows/x64/python/')
import gen3_sdk
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'   # For clarity of graph image


In [ ]:
# Status for the command streams
dict = {0: 'M2M2_APP_COMMON_STATUS_OK',
        1: 'M2M2_APP_COMMON_STATUS_ERROR',
        2: 'M2M2_APP_COMMON_STATUS_STREAM_STARTED',
        3: 'M2M2_APP_COMMON_STATUS_STREAM_STOPPED',
        4: 'M2M2_APP_COMMON_STATUS_STREAM_IN_PROGRESS',
        5: 'M2M2_APP_COMMON_STATUS_STREAM_DEACTIVATED',
        6: 'M2M2_APP_COMMON_STATUS_STREAM_COUNT_DECREMENT',
        7: 'M2M2_APP_COMMON_STATUS_STREAM_NOT_STARTED',
        8: 'M2M2_APP_COMMON_STATUS_STREAM_NOT_STOPPED',
        9: 'M2M2_APP_COMMON_STATUS_SUBSCRIBER_ADDED',
       10: 'ADI_SDK_PACKET_TIMED_OUT'}


#  <font color='brown'>TX Callback </font>

 The SDK requires the user provide functions for transmitting to and receiving data from the hardware.

Transmission is done via this callback function – which is called by the SDK when data is to be transmitted to the watch.

SDK will form the m2m2 packets and will call this API to transmit the data over the physical layer 


In [ ]:
class serial_tx_cb(gen3_sdk.watch_phy_callback):
    def __init__(self, serial_object):
        gen3_sdk.watch_phy_callback.__init__(self)
        self.serial = serial_object

    def call(self, data):
        self.serial.write(data)
        
    def sys_alert_call(self,alert):
        print("SDK ALERT : {}".format(dict[alert]))

#  <font color='brown'> RX Callback </font>

For reception, a thread is generally used to receive data from the physical layer.

When the m2m2 packets is fully received from the physical layer, it is dispatched to the SDK.

In [ ]:
class serial_rx_cb():
    def __init__(self, serial_obj):
        self.serial = serial_obj
        self.thread_run = True
        self.thread_rx = True
        self.rx_thread = threading.Thread(target=self.rx_fn)
        self.rx_thread.setDaemon(True)
        self.rx_thread.start()

    def  close(self):
        self.thread_run = False
        self.thread_rx = False

    def rx_fn(self):
        while (self.thread_rx):
            try:
                hdr = ser.read(8)
                if (hdr) :
                    length = (hdr[4] << 8) + (hdr[5])
                    #print(length)
                    body = ser.read((length ) - 8)
                    pkt = hdr + body
                    active_watch.dispatch(pkt)
            except serial.SerialException as e:
                print(e)

    def thread_start(self):
        self.serial_rx_thread.start()

#  <font color='brown'> Serial Connection</font>

User need to connect the VSM watch with our system for communication

So here we list down the number of serial ports that are available for connection.

Enter the correct serial port of the watch for communication.


In [ ]:
# Find a list of available serial ports
ser = serial.Serial()
result = []
if sys.platform.startswith('win'):
    ports = ['COM%s' % (i + 1) for i in range(256)]
elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
    # this excludes your current terminal "/dev/tty"
    ports = glob.glob('/dev/tty[A-Za-z]*')
elif sys.platform.startswith('darwin'):
    ports = glob.glob('/dev/tty.*')
else:
    raise EnvironmentError('Unsupported platform')
for port in ports:
    try:
        s = serial.Serial(port)
        s.close()
        result.append(port)
    except (OSError, serial.SerialException):
        pass

print("Available serial ports are:")
for p in result:
    print("==> {}".format(p))
    
def connect(arg):
    try:
        if(ser.isOpen()== True):
            print("Port Already Connected,Please disconnect and try again")

        print('Connecting to Motherboard...')
        ser.baudrate =921600
        ser.port = arg
        ser.open()
        print('Connected to Motherboard: ' + ser.port)

    except serial.serialutil.SerialException as e:
        print("Error opening the serial device!")
        print("The port might be already selected,or have given an incorrect serial device identifier.")
        print("Error was:\n\t{}".format(e))
        return
    
if sys.platform.startswith('win'):
    port = input("Enter the port (ex COM30) and press Enter: ")
elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
    # this excludes your current terminal "/dev/tty"
    port = input("Enter the port (ex /dev/tty.usbserial-DM3HUW9M) and press Enter: ")
elif sys.platform.startswith('darwin'):
    port = input("Enter the port (ex /dev/tty.usbserial-DM3HUW9M) and press Enter: ")
else:
    raise EnvironmentError('Unsupported platform')
connect(port)
rx_cb = serial_rx_cb(ser)

#  <font color='brown'> Creating SDK instance</font>

Create an instance for the watch class with passing the transmission callback through the function.

Then set the platform of the watch to python

In [ ]:
active_watch = gen3_sdk.watch(serial_tx_cb(ser).__disown__())
active_watch.set_platform(gen3_sdk.watch.python)

#  <font color='brown'> ECG Stream Callback</font>

SDK will provide the streams in the callback function. 

So before starting ECG streams, we need to initialize the ecg_cb callback to receive the ECG data streams.

In [ ]:
ecg_arr = []
class ecg_cb(gen3_sdk.ecg_stream_callback):
    def __init__(self):
        gen3_sdk.ecg_stream_callback.__init__(self)
        ecg_arr = []

    def call(self, data, sequence_num):
        for d in data:
            ecg_arr.append(d.ecg_data)

#  <font color='brown'> Commands to start ECG</font>


There are two command structure for starting any streams,

  1) L1 commands - Each L1 command is mapped to an equivalent M2M2 command
        
  Ex :<font color='olive'>active_watch.ecg_app.ecg_stream.subscribe(ecg_cb().__disown__())
    
   active_watch.ad5940_app.start() 
    
   active_watch.ecg_app.ecg_stream.start() </font>
                 
  2) L2 commands - L2 commands encapsulate a group of L1 commands to make using the API more easier.
        
  Ex :<font color='olive'> active_watch.start_ecg(ecg_cb().__disown__()) </font>

In [ ]:

t_end = time.time() + 10

if(ser.isOpen() == True):
        
    print("ECG stream started")
    
    active_watch.start_ecg(ecg_cb().__disown__()) 

    print("Waiting for the samples to be Collected")
    time.sleep(20)

    active_watch.stop_ecg()

    print("ECG stream stopped")
else:
    print("Serial port is not connected")



#  <font color='brown'> Plotting</font>

Plotting the ECG data

In [ ]:

samples_ecg= []
for index in range(len(ecg_arr)):
    samples_ecg.append(index)
        
#########################__PLOTTING_FOR_ECG__####################
plt.plot(samples_ecg,ecg_arr,label = "ECG", color = "red")
plt.title('ECG')
plt.ylabel('amplitude')
plt.xlabel('Samples')
plt.legend()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()
        

#  <font color='brown'> Disconnect</font>

Disconnect the serial port and stop the receiver thread.

In [ ]:
rx_cb.close()
ser.close()
print("Disconnected")